# Parse {year,type,county} subtotals from NJSP PDFs

NJSP pages for each year (example: [2023](https://www.nj.gov/njsp/info/fatalacc/2023-stats.shtml)) include links to two PDFs:
- Year to Date Comparative ([example](https://www.nj.gov/njsp/info/fatalacc/pdf/swfcs2_23.pdf))
- Victim Classification by County ([example](https://www.nj.gov/njsp/info/fatalacc/pdf/ptccr_23.pdf))

These provide some info that's not available elsewhere:
- {year, victim type, county} subtotals
- {year, victim type, age range} subtotals

These are available for all but 4 years since 2008 (the earliest NJSP data): 2008, 2009, 2017, and 2018. Later in this notebook we recover {year,type} subtotals (sans "county" facet) for those years. [read_gpt_csvs.ipynb](../annual-reports/year-type-county/read_gpt_csvs.ipynb) also recovers {year,type,county} stats for those years.

NJSP's per-crash records (`data/FAUQStats*.xml` files) only include "victim type" info since 2020, so this notebook backfills {year,type} subtotals for 2008-2019. It produces `data/year_types.csv`, which `njsp-plots.ipynb` uses as part of a daily Github Action.

In [1]:
from utz import *
from tabula import read_pdf
from njsp.paths import CRASHES_PQT, ANNUAL_SUMMARIES, ANNUAL_SUMMARIES_YT_CSV, ANNUAL_SUMMARIES_YTC_CSV, MISSING_YTC

Tabula helpers

In [2]:
def load_rects(tpl_name):
    tpl_path = f'{ANNUAL_SUMMARIES}/{tpl_name}.json'
    with open(tpl_path, 'r') as f:
        tpl = json.load(f)
    return tpl

def load_pdf_tbl(rect, pdf_path):
    [tbl] = read_pdf(pdf_path, area=[ rect[k] for k in [ 'y1', 'x1', 'y2', 'x2', ] ], pages='all',)
    return tbl

## Load "Victim Classification by County" data

In [3]:
[ptccr_rect] = load_rects('ptccr_23.tabula-template')
def load_ptccr(year):
    pdf_path = f'{ANNUAL_SUMMARIES}/ptccr_%02d.pdf' % (year % 100)
    tbl = load_pdf_tbl(ptccr_rect, pdf_path)
    tbl['year'] = year
    tbl = tbl.set_index('County')
    return tbl

In [4]:
cur_year = now().year
cur_year

2024

In [5]:
%%time
start_year = 2008
missing_years = [ 2008, 2009, 2017, 2018 ]
summaries = pd.concat([
    load_ptccr(year)
    for year in range(start_year, cur_year)
    if year not in missing_years
])
summaries.columns = summaries.columns.str.lower()
summaries = summaries.rename(columns={
    'pedalcyclist': 'cyclist',
})
summaries.index.name = 'county'
summaries.reset_index().set_index(['year', 'county'])
summaries

CPU times: user 26.3 ms, sys: 39.8 ms, total: 66 ms
Wall time: 8.95 s


,driver,passenger,cyclist,pedestrian,fatalities,crashes,year
county,,,,,,,
Atlantic,10,7,1,6,24,22,2010
Bergen,17,5,0,15,37,36,2010
Burlington,22,7,0,5,34,33,2010
Camden,17,11,3,10,41,37,2010
Cape May,4,1,0,0,5,4,2010
...,...,...,...,...,...,...,...
Somerset,14,4,0,6,24,22,2023
Sussex,6,2,0,1,9,6,2023
Union,13,6,2,15,36,34,2023


Verify "fatalities" is the sum of the 4 types:

In [6]:
type_cols = ['driver', 'passenger', 'cyclist', 'pedestrian']
assert (summaries[type_cols].sum(1) == summaries.fatalities).all()

Verify "Total" (per year) rows:

In [7]:
assert (summaries.drop(index='Total').groupby('year').sum() == summaries.set_index('year', append=True).loc['Total']).all().all()

In [8]:
missing_ytc = read_csv(MISSING_YTC).set_index(['year', 'county']).astype(int)
missing_ytc

crashes  driver  passenger  pedestrian  cyclist
year county                                                     
2008 Atlantic         30      17          8           6        0
     Bergen           22      10          5           7        1
     Burlington       45      23          6          12        4
     Camden           42      25          4          15        0
     Cape May         11       8          3           0        0
     Cumberland       23      15          5           2        1
     Essex            43      23         11          14        1
     Gloucester       29      14         13           5        1
     Hudson           24      13          4           6        4
     Hunterdon         9       6          2           3        0
     Mercer           20      11          6           4        1
     Middlesex        42      28          9           8        1
     Monmouth         45      29          9           7        2
     Morris           25      15          2           6        3
     Ocean            37      20          8          10        0
     Passaic          25       9          5          11        0
     Salem            14      13          3           0        0
     Somerset         15       8          2           5        0
     Sussex           14      12          2           0        0
     Union            35      17          4          16        1
     Warren            5       4          1           1        0
2009 Atlantic         41      22          8          12        1
     Bergen           25      14          0          12        0
     Burlington       43      27         14           7        0
     Camden           37      24          7           9        2
     Cape May          8       7          0           1        0
     Cumberland       27      21          5           1        1
     Essex            40      14          7          21        0
     Gloucester       22      14          4           3        1
     Hudson           22       7          3          12        0
     Hunterdon         8       5          1           2        0
     Mercer           21      11          4           7        0
     Middlesex        44      19         10          18        1
     Monmouth         34      18          2          11        3
     Morris           18      12          3           3        0
     Ocean            57      32         12          15        4
     Passaic          23      16          5           5        0
     Salem            13      11          1           1        1
     Somerset         13       9          3           1        0
     Sussex            7       5          2           0        0
     Union            38      21          6          14        0
     Warren            9       6          1           2        0
2017 Atlantic         34      21          5          10        0
     Bergen           25      13          5           8        1
     Burlington       46      31          5          12        0
     Camden           40      22          6          15        1
     Cape May         15       9          5           2        0
     Cumberland       24      16          4           5        1
     Essex            39      14          3          22        1
     Gloucester       43      25          9           9        1
     Hudson           23       6          2          15        3
     Hunterdon         8       7          0           1        0
     Mercer           26      13          2          11        0
     Middlesex        44      26          7          13        2
     Monmouth         40      25          6          11        1
     Morris           26      16          5           7        1
     Ocean            52      28          8          13        4
     Passaic          18       9          4           5        0
     Salem            15      16          1           0        0
     Somerset         22      13          2         

In [9]:
ytc = pd.concat([
    summaries.drop(index='Total').drop(columns='fatalities').reset_index().set_index(['year', 'county']),
    missing_ytc,
]).sort_index()
ytc

driver  passenger  cyclist  pedestrian  crashes
year county                                                     
2008 Atlantic        17          8        0           6       30
     Bergen          10          5        1           7       22
     Burlington      23          6        4          12       45
     Camden          25          4        0          15       42
     Cape May         8          3        0           0       11
...                 ...        ...      ...         ...      ...
2023 Salem            8          2        0           2       11
     Somerset        14          4        0           6       22
     Sussex           6          2        0           1        6
     Union           13          6        2          15       34
     Warren           8          1        0           3       12

[336 rows x 5 columns]

In [10]:
ytc.reset_index(level=1).county.value_counts()

Atlantic      16
Middlesex     16
Union         16
Sussex        16
Somerset      16
Salem         16
Passaic       16
Ocean         16
Morris        16
Monmouth      16
Mercer        16
Bergen        16
Hunterdon     16
Hudson        16
Gloucester    16
Essex         16
Cumberland    16
Cape May      16
Camden        16
Burlington    16
Warren        16
Name: county, dtype: int64

In [11]:
ytc.to_csv(ANNUAL_SUMMARIES_YTC_CSV)

In [12]:
yt = ytc.reset_index(level=1, drop=True).groupby(lambda x:x).sum()
yt

,driver,passenger,cyclist,pedestrian,crashes
year,,,,,
2008,320,112,20,138,555
2009,315,98,14,157,550
2010,303,99,13,141,530
2011,362,105,17,143,586
2012,309,103,14,163,553
2013,304,92,14,132,508
2014,295,80,11,170,523
2015,276,96,17,173,522
2016,330,89,17,166,570


In [13]:
yt.to_csv(ANNUAL_SUMMARIES_YT_CSV)

## Compare to per-crash records
NJSP's per-crash datasets include victim-type info for crashes since 2020.

This allows for a sanity-check of the {year,type,county} totals reported in the "Victim Classification by County" PDFs (for 2020-2023):

In [14]:
sp = read_parquet(CRASHES_PQT)
sp

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,INJURIES,STREET,FATAL_D,FATAL_P,FATAL_T,FATAL_B,dt
ACCID,,,,,,,,,,,,,,
1703,01,Atlantic,0102,Atlantic City,446,State/Interstate Authority 446 S MP 1,1.0,1.0,None,NaN,NaN,NaN,NaN,2008-01-01 00:35:00
1681,09,Hudson,0910,Union City,None,Bergenline Ave S MP 0 at 6th St,1.0,NaN,Bergenline Ave,NaN,NaN,NaN,NaN,2008-01-01 04:11:00
1659,04,Camden,0415,Gloucester Twsp,42,State Highway 42 N MP 8.2,1.0,1.0,None,NaN,NaN,NaN,NaN,2008-01-01 06:46:00
1661,20,Union,2004,Elizabeth City,624,County 624 W MP 2.2 at Ikea Dr,1.0,1.0,None,NaN,NaN,NaN,NaN,2008-01-01 12:29:00
1811,07,Essex,0716,Nutley Town,648,County 648 E MP .87 at Franklin Ave,1.0,NaN,None,NaN,NaN,NaN,NaN,2008-01-01 18:53:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12940,02,Bergen,0219,Fort Lee Boro,None,Bruce Reynolds Blvd,1.0,NaN,Bruce Reynolds Blvd,0.0,0.0,1.0,0.0,2024-01-10 05:09:00
12939,01,Atlantic,0121,Somers Point City,None,Bay Ave,1.0,NaN,Bay Ave,1.0,0.0,0.0,0.0,2024-01-10 11:08:00
12942,15,Ocean,1506,Brick Twsp,None,nj 35/sea breeze way,1.0,NaN,nj 35/sea breeze way,0.0,0.0,1.0,0.0,2024-01-13 09:17:00


In [15]:
cols = [ 'FATALITIES', 'STREET', 'FATAL_D', 'FATAL_P', 'FATAL_T', 'FATAL_B', ]
y = sp.dt.dt.year.rename('year')
c = sp.CNAME.rename('county')
gb = sp.groupby([y, c])
agg = gb[cols].sum(numeric_only=True).astype(int)
agg['crashes'] = gb.size()
agg = agg.rename(columns={
    'FATALITIES': 'fatalities',
    'FATAL_D': 'driver',
    'FATAL_P': 'passenger',
    'FATAL_T': 'pedestrian',
    'FATAL_B': 'cyclist',
})
agg20 = agg.reset_index(level=1)
agg20 = agg20[agg20.index >= 2020]
assert (agg20[type_cols].sum(1) == agg20.fatalities).all()
agg = agg.drop(columns='fatalities')
agg

driver  passenger  pedestrian  cyclist  crashes
year county                                                     
2008 Atlantic         0          0           0        0       30
     Bergen           0          0           0        0       22
     Burlington       0          0           0        0       45
     Camden           0          0           0        0       42
     Cape May         0          0           0        0       11
...                 ...        ...         ...      ...      ...
2024 Mercer           1          0           0        0        1
     Middlesex        0          0           2        0        2
     Monmouth         0          2           1        0        3
     Ocean            0          0           3        0        3
     Union            0          0           1        0        1

[343 rows x 5 columns]

### Combine aggregate stats from per-crash records vs. stats from PDFs

In [16]:
m = agg20.reset_index()
m = m.merge(summaries.reset_index(), how='left', on=['year', 'county'], suffixes=['_sp', '_stats']).dropna()
m = m.set_index(['year', 'county'])
m = m[m.columns.sort_values()].astype(int)
m.columns = pd.MultiIndex.from_tuples([ tuple(reversed(col.split('_'))) for col in m.columns ])
m = m[m.columns.sort_values()].astype(int)
m

sp                                                 \
                crashes cyclist driver fatalities passenger pedestrian   
year county                                                              
2020 Atlantic        38       0     26         40         5          9   
     Bergen          38       0     14         43         9         20   
     Burlington      40       3     26         42         4          9   
     Camden          36       1     19         38         5         13   
     Cape May         8       1      5          9         0          3   
     Cumberland      22       0     14         24         5          5   
     Essex           39       3     16         45        12         14   
     Gloucester      33       2     21         35         5          7   
     Hudson          24       1     11         24         1         11   
     Hunterdon       12       0      7         12         2          3   
     Mercer          20       0      9         21         5          7   
     Middlesex       61       5     27         67        12         23   
     Monmouth        33       0     23         34         2          9   
     Morris          17       1      8         17         2          6   
     Ocean           30       1     20         30         5          4   
     Passaic         26       0     17         29         5          7   
     Salem           11       0      6         14         5          3   
     Somerset        14       0      7         14         0          7   
     Sussex           7       0      6          7         0          1   
     Union           29       0     13         30         1         16   
     Warren          12       0      9         12         1          2   
2021 Atlantic        46       2     22         49         6         19   
     Bergen          35       0     20         35         2         13   
     Burlington      40       2     24         41         4         11   
     Camden          67       2     30         69         2         35   
     Cape May        12       3      4         12         3          2   
     Cumberland      27       2     17         28         2          7   
     Essex           60       3     22         61         5         31   
     Gloucester      42       1     28         48        10          9   
     Hudson          29       3     11         30         3         13   
     Hunterdon        9       0      9          9         0          0   
     Mercer          35       2     16         38         8         12   
     Middlesex       45       1     26         47         8         12   
     Monmouth        32       1     18         34         4         11   
     Morris          25       1     15         25         5          4   
     Ocean           43       0     26         44         9          9   
     Passaic         24       1     12         25         4          8   
     Salem           20       0     19         22         2          1   
     Somerset        25       0     21         25         1          3   
     Sussex          11       1      7         11         0          3   
     Union           27       1     11         29         6         11   
     Warren          13       0     10         15         2          3   
2022 Atlantic        34       2     18         34         3         11   
     Bergen          38       0     21         44        10         13   
     Burlington      54       1     34         60        12         13   
     Camden          51       1     22         54         5         26   
     Cape May        14       1      5         15         1          8   
     Cumberland      20       0     15         20         1          4   
     Essex           50       0     26         56         9         21   
     Gloucester      39       0     28         40         5          7   
     Hudson          15       0      9         17         2          6   
     Hunterdon        8  

In [17]:
row_diffs = (m['sp'] == m['stats']).all(1)
row_diffs = row_diffs[~row_diffs]
row_diffs

Series([], dtype: bool)

In [18]:
assert row_diffs.all(), row_diffs